In [39]:
import pandas.util.testing as tm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
import folium
from folium.plugins import HeatMap

In [45]:
df = pd.read_csv('US_accident_cleaned')
state='GA'

df_ga=df.loc[df.State==state].copy()
df_ga.drop('State',axis=1, inplace=True)

In [46]:
feature_lst=['TMC','Severity','Start_Lng','Start_Lat','Distance(mi)','Temperature(F)','Humidity(%)','Visibility(mi)']
df_sel=df_ga[feature_lst].copy()
df_sel.to_csv('./df_sel.csv',index=False)
df_sel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62277 entries, 141801 to 2180192
Data columns (total 8 columns):
TMC               62277 non-null float64
Severity          62277 non-null float64
Start_Lng         62277 non-null float64
Start_Lat         62277 non-null float64
Distance(mi)      62277 non-null float64
Temperature(F)    62277 non-null float64
Humidity(%)       62277 non-null float64
Visibility(mi)    62277 non-null float64
dtypes: float64(8)
memory usage: 4.3 MB


In [47]:
target='Severity'

y = df_sel[target]
X = df_sel.drop(target, axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

In [48]:
# Random Forest algorithm

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

# Get the accuracy score
acc=accuracy_score(y_test, y_pred)


# Model Accuracy, how often is the classifier correct?
print("[Randon forest algorithm] accuracy_score: {:.3f}.".format(acc))

[Randon forest algorithm] accuracy_score: 0.867.


In [49]:
print(y_pred)
print(y_pred.shape)
# print(X_test)
print(X_test.shape)

[3. 3. 2. ... 3. 2. 2.]
(12456,)
(12456, 7)


In [50]:
X_test.head(10)

,TMC,Start_Lng,Start_Lat,Distance(mi),Temperature(F),Humidity(%),Visibility(mi)
154287,201.0,-84.462776,33.889336,0.00,73.0,87.0,10.0
144077,201.0,-84.378700,33.754379,0.01,30.0,82.0,10.0
1075349,201.0,-84.367554,33.820381,0.00,50.0,66.0,10.0
749764,201.0,-82.047447,33.260040,0.00,76.0,91.0,3.0
840025,201.0,-84.439552,33.864948,2.35,83.0,49.0,6.0
574363,201.0,-82.026619,33.434250,0.00,68.0,93.0,8.0
153078,201.0,-84.230499,33.506004,0.00,89.1,40.0,10.0
1323761,201.0,-84.495628,33.753635,0.00,54.0,80.0,10.0
2036173,241.0,-84.005554,33.653748,0.00,66.0,100.0,4.0
148262,201.0,-84.560516,33.773994,0.00,84.9,49.0,10.0


In [51]:
X_test['Severity']=y_pred
df_sel = X_test
X_test.head(10)

/Users/xiaoyingyang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,TMC,Start_Lng,Start_Lat,Distance(mi),Temperature(F),Humidity(%),Visibility(mi),Severity
154287,201.0,-84.462776,33.889336,0.00,73.0,87.0,10.0,3.0
144077,201.0,-84.378700,33.754379,0.01,30.0,82.0,10.0,3.0
1075349,201.0,-84.367554,33.820381,0.00,50.0,66.0,10.0,2.0
749764,201.0,-82.047447,33.260040,0.00,76.0,91.0,3.0,2.0
840025,201.0,-84.439552,33.864948,2.35,83.0,49.0,6.0,3.0
574363,201.0,-82.026619,33.434250,0.00,68.0,93.0,8.0,2.0
153078,201.0,-84.230499,33.506004,0.00,89.1,40.0,10.0,3.0
1323761,201.0,-84.495628,33.753635,0.00,54.0,80.0,10.0,3.0
2036173,241.0,-84.005554,33.653748,0.00,66.0,100.0,4.0,3.0
148262,201.0,-84.560516,33.773994,0.00,84.9,49.0,10.0,3.0


In [52]:
traffic_df = df_sel.groupby(['Start_Lng','Start_Lat'])['Severity'].count()
traffic_df =traffic_df.to_frame()
traffic_df.columns.values[0]='count1'
traffic_df = traffic_df.reset_index()
coor = traffic_df[['Start_Lat','Start_Lng','count1']].values.tolist()
    
hmap = folium.Map(location=[min(df_sel['Start_Lat']),min(df_sel['Start_Lng'])], zoom_start=10, )
hmap.add_child(HeatMap(coor, radius = 5))
hmap